# Hyperparameter Search

## 1. Load packages

In [1]:
! pip install kornia==0.5.4 lightweight-gan pytorch_fid

In [2]:
from glob import glob
import os
import itertools
import numpy as np
import shutil
import re

## 2. Load the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Change the directory
os.chdir('/content/drive/My Drive/CS Courses/CS156/Assignments/Final')

## 3. Generate random sample of 500 images

In [5]:
# #Randomly pick 500 images to copy
# images_to_copy = np.random.choice(glob("cityscapes_resized_256/*"),500)

# #Iterate through all the selected images and copy them to file
# for src_file in images_to_copy:
#   dst_file = src_file.replace("cityscapes_resized_256", "real_images_500")
#   shutil.copy(src_file, dst_file)

In [6]:
len(glob("hyperparam_search_images/*"))

1312

## 4. Specify parameters for grid search

In [31]:
#Note down the parameter values to try
batch_size = [10,16]
learning_rate = [2e-4,0.001]
attn_layers = [[32],[]]
disc_output_size = [1,5]

#Get all possible configurations of the parameters
params = [batch_size,learning_rate,attn_layers,disc_output_size]
param_configs = list(itertools.product(*params))

In [32]:
#List to store grid search results
grid_search_results = []

#Iterate through all the configs
for (idx,(a,b,c,d)) in enumerate(param_configs):

  run_name = f"config_{idx+1}"

  ! lightweight_gan \
      --data ./cityscapes_resized_256 \
      --name {run_name} \
      --image-size 256 \
      --batch-size {a} \
      --learning_rate {b} \
      --gradient-accumulate-every 4 \
      --num-train-steps 100 \
      --save_every 100 \
      --evaluate_every 100 \
      --attn-res-layers {c} \
      --disc_output_size {d} \
      --amp
  
  ! lightweight_gan \
    --name {run_name} \
    --load-from 1 \
    --generate \
    --num-image-tiles 1312

  path = f"results/{run_name}-generated-1"

  total_results = !python -m pytorch_fid {path} hyperparam_search_images
  fid_string = total_results[-1]
  print(fid_string)
  fid = float(re.search(r'\d+.?\d+', fid_string)[0])

  grid_search_results.append({'batch_size':a, 'lr':b, 'attn_layers':c, 
                              'disc_output_size':d, 'fid': fid})


/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,
continuing from previous epoch - 1
loading from version 0.20.5
autosetting augmentation probability to 28%
config_1<./cityscapes_resized_256>: 100% 100/100 [00:00<?, ?it/s]
/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,
loading from version 0.20.5
Saving generated default images: 100% 1312/1312 [00:19<00:00, 67.11it/s]
Saving generated EMA images: 100% 1312/1312 [00:18<00:00, 70.02it/s]
sample images generated at /content/drive/My Drive/CS Courses/CS156/Assignments/Final/results/config_1-generated-1
FID:  408.8257595674637
/usr/local

In [35]:
import pandas as pd
grid_search_df = pd.DataFrame(grid_search_results)

In [36]:
grid_search_df

,batch_size,lr,attn_layers,disc_output_size,fid
0,10,0.0002,[32],1,408.825760
1,10,0.0002,[32],5,403.373073
2,10,0.0002,[],1,442.251676
3,10,0.0002,[],5,424.140485
4,10,0.0010,[32],1,270.030755
5,10,0.0010,[32],5,317.658936
6,10,0.0010,[],1,285.804233
7,10,0.0010,[],5,352.565193
8,16,0.0002,[32],1,379.513304
9,16,0.0002,[32],5,403.530954


In [37]:
grid_search_df.to_csv("grid_search_results.csv",index=False)